In [1]:
#### Environment ###

#from platform import python_version
#print(python_version())

#pip install tensorflow

#pip install keras

#pip install sklearn


In [2]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import numpy as np
import pandas as pd
from keras import metrics
import keras
import tensorflow as tf
import os

In [3]:
def split_data(index, train_percent):
    
    filename = "./tagged/" + str(index) + "_tagged.csv"
    data = pd.read_csv(filename)
    data_len = data.shape[0]
    
    
    
    
    data = data.drop(columns=['date'])

    print("Total size: ", data.shape[0])
    train_size = int(data_len * train_percent)
    print("train_size:",int(train_size))
    
    train_dataframe = data.loc[:train_size - 1,:]
    
    
    valid_dataframe = data.loc[train_size:,:]
    
    
    train_array = train_dataframe.to_numpy()
    valid_array = valid_dataframe.to_numpy()
    
    trainX = []
    trainY = []
    validX = []
    validY = []
    
    for value in train_array:
        trainX.append(value[:-1])
        trainY.append(value[-1])
        
    for value in valid_array:
        validX.append(value[:-1])
        validY.append(value[-1])
    
    features = array(data.columns)[:-1]
    #print(array(data.columns)[-1])
    #print(features)
    return trainX, trainY, validX, validY, features
    
#     trainY = np.array(trainY).reshape((-1, len(trainY), 1))
#     trainX = np.array(trainX).reshape((-1, len(trainX), len(trainX[0])))
    
#     validY = np.array(validY).reshape((-1, len(validY), 1))
#     validX = np.array(validX).reshape((-1, len(validX), len(validX[0])))

    
#     return trainX, trainY, validX, validY, features

trainX, trainY, validX, validY, features = split_data(0, 0.7)


Total size:  1064
train_size: 744


In [4]:
def train_model(trainX, trainY):
    
    from keras.layers import Dropout

    trainX = np.array(trainX)
    y_train = np.array(trainY)

    #Reshape the data into 3-D array
    x_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))

    # Initialising the RNN
    model = Sequential()

    model.add(LSTM(units = 50, return_sequences = True, input_shape = (trainX.shape[1], 1)))
    model.add(Dropout(0.2))

    # Adding a second LSTM layer and Dropout layer
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    # Adding a third LSTM layer and Dropout layer
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    # Adding a fourth LSTM layer and and Dropout layer
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))

    # Adding the output layer
    # For Full connection layer we use dense
    # As the output is 1D so we use unit=1
    model.add(Dense(units = 1))


    #compile and fit the model on 30 epochs
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    model.fit(x_train, y_train, epochs = 30, batch_size = 50)
    
    return model

def predict_valid(model, validX, validY):

    #Convert x_test to a numpy array 
    x_test = np.array(validX)
    y_test = np.array(validY)

    #Reshape the data into 3-D array
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

    #check predicted values
    predictions = model.predict(x_test) 
    
    # #Undo scaling
    # predictions = scaler.inverse_transform(predictions)

     #Calculate RMSE score
    rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
    print("Accuracy is ",rmse)
    return predictions
    
def predict(model, testX):
    x_test = np.array(validX)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
    predictions = model.predict(x_test) 
    return predictions



In [5]:
my_model = train_model(trainX, trainY)
predictions = predict_valid(my_model, validX, validY)

Epoch 1/30
15/15 [==============================] - 11s 400ms/step - loss: 0.9913
Epoch 2/30
15/15 [==============================] - 6s 394ms/step - loss: 1.0213
Epoch 3/30
15/15 [==============================] - 8s 529ms/step - loss: 1.1972
Epoch 4/30
15/15 [==============================] - 7s 486ms/step - loss: 0.9309
Epoch 5/30
15/15 [==============================] - 7s 468ms/step - loss: 0.9622
Epoch 6/30
15/15 [==============================] - 7s 441ms/step - loss: 1.1010
Epoch 7/30
15/15 [==============================] - 6s 423ms/step - loss: 1.0263
Epoch 8/30
15/15 [==============================] - 6s 430ms/step - loss: 1.1261
Epoch 9/30
15/15 [==============================] - 6s 429ms/step - loss: 1.2424
Epoch 10/30
15/15 [==============================] - 6s 429ms/step - loss: 0.8827
Epoch 11/30
15/15 [==============================] - 7s 441ms/step - loss: 1.0686
Epoch 12/30
15/15 [==============================] - 7s 438ms/step - loss: 1.0453
Epoch 13/30
15/15 [=====